# Visualizing the $n$th root of imaginary numbers

In [ ]:
import os
import itertools
import numpy as np

import matplotlib
import matplotlib.cm as cm
import matplotlib.pyplot as plt

from matplotlib.patches import Circle, Wedge

from adjustText import adjust_text

## Create square-shaped base figure without annotations

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))
ax.axis('off')

plt.show()

## Add coordinates axes

In [ ]:
def add_coordinate_axes(ax, *, lim=1.0, head_proportion=1/15,
                        color='0.7', lw=1.0, ls='-') -> None:
    """
    Adds light gray X and Y coordinate axes to the plot.

    Parameters:
    -----------
    lim : float
        Extent of coordinate axes in the positive and negative direction
        for both X and Y axes

    head_proportion : float
        Proportionate size of the arrow head compared to the size of the
        `lim` variable. Smaller values produce smaller arrow heads.
        Setting this value produces a consistent arrow head size for
        plots, regardless of the value of `lim`.

    color: float or str
        Color of the drawn line.

    lw: float
        Width of the drawn line.

    ls : str or tuple
        Linestyle definition compatible with `matplotlib`'s
       `matplotlib.lines.Line2D.set_linestyle()` method. Style of the
       drawn lines.
    """
    head_size = lim * head_proportion

    # X coordinate axis
    ax.arrow(x=-lim, y=0, dx=2*lim, dy=0,
             color=color, length_includes_head=True,
             head_width=head_size, head_length=head_size)
    ax.text(x=0.9*lim, y=-0.2*lim, s="Re",
            color=color, fontsize=25, fontweight='bold')
    # Y coordinate axis
    ax.arrow(x=0, y=-lim, dx=0, dy=2*lim,
             color=color, length_includes_head=True,
             head_width=head_size, head_length=head_size)
    ax.text(x=-0.25*lim, y=0.9*lim, s="Im",
            color=color, fontsize=25, fontweight='bold')
    return

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))
ax.axis('off')

# Add coordinate axes
add_coordinate_axes(ax, lim=2.5, head_proportion=1/15,
                    color='0.7', lw=1.0, ls='-')

plt.show()

## Add vector of arbitrary magnitude $r$ and phase $\vartheta$ along with a circle of radius $r$

In [ ]:
def add_vector(ax, *, r=1.0, theta=1.0,
               color=None, lw=1.5, ls='--') -> tuple:
    """
    Adds a vector of arbitrary magnitude and phase to the plot. Adds a
    blob marker at its end's position. Annotates the vector with a given
    text annotation.

    Parameters:
    -----------
    r : float
        Magnitude of the arbitrary vector.

    theta : float
        Phase of the arbitrary vector in radians.

    color: float or str
        Color of the drawn line.

    lw: float
        Width of the drawn line.

    ls : str or tuple
        Linestyle definition compatible with `matplotlib`'s
       `matplotlib.lines.Line2D.set_linestyle()` method. Style of the
       drawn lines.
    """
    # Calculate coordinates of the endpoint of the vector
    x1 = [0, 0]
    x2 = [r*np.cos(theta), r*np.sin(theta)]

    # Draw the vector
    l = ax.plot((x1[0], x2[0]), (x1[1], x2[1]),
                 color=color, lw=lw, ls=ls)

    # Blob marker at the end of the vector
    s = ax.scatter(*x2, s=10**2, color=color)

    return x1, x2, s, l

In [ ]:
def add_circle(ax, *, r=1.0,
               color='0.7', lw=1.5, ls='--') -> matplotlib.patches.Circle:
    """
    Draws a circle with an arbitrary radius on the plot centered around
    the coordinates (0, 0).

    Parameters:
    -----------
    r : float
        Radius of the circle.

    color: float or str
        Color of the drawn line.

    lw: float
        Width of the drawn line.

    ls : str or tuple
        Linestyle definition compatible with `matplotlib`'s
       `matplotlib.lines.Line2D.set_linestyle()` method. Style of the
       drawn lines.
    """
    # Draw the circle on the `ax` subplot
    c = Circle((0, 0), radius=r, fill=False,
               color=color, lw=lw, ls=ls)
    ax.add_patch(c)
    return c

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))
ax.axis('off')

# Add coordinate axes
add_coordinate_axes(ax, lim=2.5, head_proportion=1/15,
                    color='0.7', lw=1.0, ls='-')

# Add an arbitrary vector to the plot
_ = add_circle(ax, r=2.0,
               color='0.7', lw=1.5, ls='--')
_ = add_vector(ax, r=2.0, theta=1.0,
               color='0.7', lw=1.5, ls='--')

plt.show()

## Plot the $n$th roots of a vector

In [ ]:
def get_colors(n) -> list:
    # Get different colors for each new root vector from a nice colormap,
    # but let the "first" root to be colored black
    colors = np.r_[
        [[0.0, 0.0, 0.0, 1.0]],
        cm.rainbow(np.linspace(0, 1, n-1))
    ]
    return colors

In [ ]:
def plot_roots(ax, *, n, r=1.0, theta=1.0,
               lw=1.5, ls='-') -> tuple:
    """
    Parameters:
    -----------
    n : int, larger than 0
        Degree of the root extraction.

    r : float
        Magnitude of the arbitrary vector.

    theta : float
        Phase of the arbitrary vector in radians.

    lw: float
        Width of the drawn lines.

    ls : str or tuple
        Linestyle definition compatible with `matplotlib`'s
       `matplotlib.lines.Line2D.set_linestyle()` method. Style of the
       drawn lines.
    """
    assert n > 0 and isinstance(n, int), "`n` should be a larger integer, than 0!"

    r_root = np.power(r, 1/n)  # Magnitude of the root vectors
    colors = get_colors(n)     # Colors of the root vectors
    xs = []                    # Storage for root vector end points
    scats = []                 # Storage for scatter elements
    lines = []                 # Storage for line elements

    # Iterate over all root vectors and plot them
    for r_i in range(n):
        # Phase of the next root vector
        theta_root = (theta + r_i * 2*np.pi) / n
        # Plot the next root vector
        _, x, s, l = add_vector(ax, r=r_root, theta=theta_root,
                                color=colors[r_i], lw=lw, ls=ls)
        xs += [x]
        scats += [s]
        lines += [l]
    return xs, scats, lines

In [ ]:
def draw_angle(ax, *, n, r=1.0, theta=1.0) -> matplotlib.patches.Wedge:
    """
    Parameters:
    -----------
    n : int, larger than 0
        Degree of the root extraction.

    r : float
        Magnitude of the arbitrary vector.

    theta : float
        Phase of the arbitrary vector in radians.
    """
    assert n > 0 and isinstance(n, int), "`n` should be a larger integer, than 0!"

    r_root = np.power(r, 1/n)  # Magnitude of the root vectors
    theta_root = theta / n     # Phase of the first root vector

    # Plot the angle between the real axis and the first root vector
    w = Wedge((0, 0), r=r_root, theta1=0.0, theta2=np.rad2deg(theta/n),
              color='0.8', zorder=0)
    ax.add_patch(w)

    # Draw phi/n on the `Wedge` patch
    p = (
        0.85 * r_root*np.cos(theta_root/2),
        0.85 * r_root*np.sin(theta_root/2)
    )
    text = ax.text(*p, f"$\\mathbf{{\\varphi/{n}}}$",
                   ha='center', va='center', fontsize=12)
    return w, text

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))
ax.axis('off')

# Add coordinate axes
add_coordinate_axes(ax, lim=2.5, head_proportion=1/15,
                    color='0.7', lw=1.0, ls='-')

# Add an arbitrary vector to the plot
_ = add_circle(ax, r=2.0,
               color='0.7', lw=1.5, ls='--')
_ = add_vector(ax, r=2.0, theta=1.0,
               color='0.7', lw=1.5, ls='--')

# Add roots of the vector to the plot
n = 4
_ = add_circle(ax, r=np.power(2.0, 1/n),
               color='0.0', lw=1.5, ls='-')
_ = plot_roots(ax, n=n, r=2.0, theta=1.0,
               lw=1.5, ls='-')

# Mark the phase of the first root vector
_ = draw_angle(ax, n=n, r=2.0, theta=1.0)

plt.show()

## Self-adjusting annotations for the vectors on the figure

In [ ]:
def annotate(ax, *, pos, s='', **kwargs) -> matplotlib.text.Text:
    """
    Wrapper for the `matplotlib.pyplot.text()` method.
    """
    text = ax.text(*pos, s=s, va='center', ha='center', **kwargs)
    return text

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))
ax.axis('off')

# Add coordinate axes
add_coordinate_axes(ax, lim=2.5, head_proportion=1/15,
                    color='0.7', lw=1.0, ls='-')

# Add an arbitrary vector to the plot
_ = add_circle(ax, r=2.0,
               color='0.7', lw=1.5, ls='--')
_, x, s, _ = add_vector(ax, r=2.0, theta=1.0,
                        color='0.7', lw=1.5, ls='--')

# Add roots of the vector to the plot
n = 10
_ = add_circle(ax, r=np.power(2.0, 1/n),
               color='0.0', lw=1.5, ls='-')
xs, scats, lines = plot_roots(ax, n=n, r=2.0, theta=1.0,
                              lw=1.5, ls='-')

# Mark the phase of the first root vector
w, w_annot = draw_angle(ax, n=n, r=2.0, theta=1.0)


# ---- Annotation ----
# Store all vector end points for annotation
POI = np.array([x] + xs)
# Store matplotlib patches to repel text annotations from
patches = []
patches.extend([s])
patches.extend(list(itertools.chain(*[scats, *lines])))
patches.append(w)
# Store `matplotlib.text.Text` objects for the `adjustText` module
texts = []

# Colors of the root vector annotations
colors = get_colors(n)

# Annotate the original vector
annotation = '$r e^{i \\varphi}$'
texts.append(annotate(ax, pos=POI[0], s=annotation,
                      color='0.7', fontsize=18))
# Annotate the root vectors
for r_i in range(n):
    annotation = f'$\\sqrt[{n}]{{r}} e^{{i ( \\varphi + {r_i} \cdot 2 \pi ) / {n}}}$'
    texts.append(annotate(ax, pos=POI[1:][r_i], s=annotation,
                          color=colors[r_i], fontsize=18))

# Adjust annotations
adjust_text(texts, add_objects=patches, lim=1000)
if n > 6:
    adjust_text([w_annot], add_objects=patches + texts,
                arrowprops=dict(arrowstyle='->', color='black'))

plt.show()